In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/edsa-movie-recommender-challenge-2022/sample_submission.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/movies.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/imdb_data.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/genome_tags.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/genome_scores.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/train.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/test.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/tags.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/links.csv


In [2]:
import scipy as sp 
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
train=pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/train.csv')
test=pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/test.csv')
links=pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/links.csv')
imdb_data=pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/imdb_data.csv')
genome_tags=pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/genome_tags.csv')
genome_scores=pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/genome_scores.csv')
movies=pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/movies.csv')
sample_submission=pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/sample_submission.csv')

In [4]:
train.head()

,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837


In [5]:
train=train.drop('timestamp',axis=1)
train.head()

,userId,movieId,rating
0,5163,57669,4.0
1,106343,5,4.5
2,146790,5459,5.0
3,106362,32296,2.0
4,9041,366,3.0


In [6]:
test.head()

,userId,movieId
0,5,788
1,68,7438
2,336,40412
3,803,3822
4,547,903


In [7]:
sample_train=train.sample(n=5000,random_state=2022)
sample_train

,userId,movieId,rating
4407603,94511,3863,1.0
7254705,44709,610,4.0
4680707,7478,726,3.0
986738,57076,1407,3.0
5293985,39803,2324,3.5
...,...,...,...
7101630,104793,1265,5.0
7256170,49514,150,3.0
1473579,151386,2580,3.5
6743616,32705,3418,4.0


In [8]:
pivot_table=pd.pivot_table(data=sample_train,index='userId',columns='movieId',values='rating')
pivot_table

movieId,1,2,3,5,6,7,8,9,10,11,...,195427,197509,197529,197689,197711,198815,199101,200332,201773,205449
userId,,,,,,,,,,,,,,,,,,,,,
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
pivot_table.fillna(0,inplace=True)
pivot_table

movieId,1,2,3,5,6,7,8,9,10,11,...,195427,197509,197529,197689,197711,198815,199101,200332,201773,205449
userId,,,,,,,,,,,,,,,,,,,,,
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
util_matrix_norm = pivot_table.T
util_matrix_sparse = sp.sparse.csr_matrix(util_matrix_norm.values)

In [11]:
user_similarity = cosine_similarity(util_matrix_sparse.T)
user_sim_df = pd.DataFrame(user_similarity,
                           index = util_matrix_norm.columns,
                           columns = util_matrix_norm.columns)

In [12]:
user_sim_df

userId,20,226,228,267,450,483,511,561,575,581,...,162306,162350,162401,162416,162441,162452,162462,162476,162495,162508
userId,,,,,,,,,,,,,,,,,,,,,
20,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
226,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
228,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
267,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
450,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
162462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
162476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [13]:
test.head()

,userId,movieId
0,5,788
1,68,7438
2,336,40412
3,803,3822
4,547,903


In [14]:
test.head()

,userId,movieId
0,5,788
1,68,7438
2,336,40412
3,803,3822
4,547,903


In [15]:
def movie_rating(reference_user, reference_movie):
    
    if (reference_user not in user_sim_df.columns) and (reference_movie not in pivot_table.columns): #CASE 3
        movie_rating=train['rating'].mean() #Return global average of all ratings
        movie_rating=round(movie_rating*2)/2
        return movie_rating 
    elif reference_user not in user_sim_df.columns: 
        grouped_by=train[['rating']].groupby(train['movieId']==reference_movie).mean()
        movie_rating=grouped_by._get_value(1, 'rating', takeable=False)
        movie_rating=round(movie_rating*2)/2
        return movie_rating
    else:
        #GET THE SIMILARITY OF TOP 10 USERS TO THE REFERENCE USER
        user_similarity_table=user_sim_df.loc[:,reference_user]    
        user_similarity_sorted=user_similarity_table.sort_values(ascending=False)
        user_similarity_sorted.drop(index=user_similarity_sorted.index[0],axis=0, inplace=True) #drop first row
        most_similar_users=user_similarity_sorted.head(10)
        most_similar_users=most_similar_users.to_frame()
        most_similar_users
        #CASE 2: REFERENCE MOVIE NOT IN PIVOT TABLE
        if reference_movie not in pivot_table.columns:
            grouped_by_movie_rating=train[['rating']].groupby(train['userId']==reference_user).mean()
            movie_rating=grouped_by_movie_rating._get_value(1, 'rating', takeable=False)
            movie_rating=round(movie_rating*2)/2
            return movie_rating
        else:
            #GET THE RATING EACH USER GAVE TO THE REFERENCE MOVIE
            user_ratings = pivot_table.loc[:,reference_movie]  
            ratings=list()              
            most_similar_users=most_similar_users.reset_index()
            for index,row in most_similar_users.iterrows():
                user=row['userId']
                user=int(user)
                user_rating=user_ratings.loc[user]
                ratings.append(user_rating)
            #CONCAT THE MOST SIMILAR USERS AND THEIR SIMILARITY SCORE WITH THE RATINGS THEY GAVE A PARTICULAR MOVIE
            most_similar_users['sim_to_current_user']=most_similar_users[reference_user]
            most_similar_users['most_similar_users_rating_of_current_movie'] =pd.Series(ratings) 
            most_similar_users = most_similar_users.drop(reference_user, 1)
            most_similar_users['Similarity X rating']=most_similar_users['sim_to_current_user']*most_similar_users['most_similar_users_rating_of_current_movie']
            most_similar_users
            #most_similar_users
            try:
                movie_rating=sum(most_similar_users['Similarity X rating']) / sum(most_similar_users['sim_to_current_user'])
                movie_rating=round(movie_rating*2)/2
                return movie_rating
            except ZeroDivisionError:
                movie_rating=float(0)
                movie_rating=round(movie_rating*2)/2
                return movie_rating


In [16]:
final_movies_rated_list=list()
for indexes,rows in test.iterrows():
    reference_user=rows['userId']
    reference_movie=rows['movieId']
    final_movies_rated=movie_rating(reference_user, reference_movie)
    final_movies_rated_list.append(final_movies_rated)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:39: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [17]:
sample_submission.head()

,Id,rating
0,1_2011,1.0
1,1_4144,1.0
2,1_5767,1.0
3,1_6711,1.0
4,1_7318,1.0


In [18]:
submission_df=test.copy()
submission_df.head()

,userId,movieId
0,5,788
1,68,7438
2,336,40412
3,803,3822
4,547,903


In [19]:
#creating the test dataframe
submission_df['Id']=submission_df['userId'].astype(str)+'_'+submission_df['movieId'].astype(str)
submission_df['rating']=final_movies_rated_list
submission_df=submission_df.drop(['userId','movieId'],axis=1)
submission_df.head()


,Id,rating
0,5_788,3.0
1,68_7438,4.0
2,336_40412,3.5
3,803_3822,4.0
4,547_903,4.0


In [20]:
submission_df.to_csv('./predictions.csv',index=False)